In [1]:
!export HF_HOME="/scratch/ssd004/scratch/lfy"

In [2]:
from llama_cpp import Llama, LlamaGrammar
import json

In [3]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [6]:
llm = Llama.from_pretrained(
    repo_id="QuantFactory/Meta-Llama-3.1-8B-Instruct-GGUF",
    filename="*Q4_K_M.gguf",
    # n_gpu_layers=-1,
    n_ctx=128000,
    verbose=True,
    cache_dir="/checkpoint/lfy/14024697"
)

llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from /checkpoint/lfy/14024697/models--QuantFactory--Meta-Llama-3.1-8B-Instruct-GGUF/snapshots/b6d5cca03f341fd97b7657420bd60e070835b7e5/./Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3.1
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam

In [7]:
ask_food_prompt_template = "The user is living in {user_location}. The user is looking for a {user_query}. Suggest 5 {user_query} which can be cooked by the user, without actual recipe."

In [8]:
user_location = "California"
user_query = "vegan korean food"
ask_food_prompt = ask_food_prompt_template.format(user_location=user_location, user_query=user_query)

In [9]:
schema_food_name = r'''
root ::= (
    "{" newline
        doublespace "\"food_names\":" space listofstring newline
    "}"
)
newline ::= "\n"
doublespace ::= "  "
number ::= [0-9]+   "."?   [0-9]*
boolean ::= "true" | "false"
char ::= [^"\\\x7F\x00-\x1F] | [\\] (["\\bfnrt] | "u" [0-9a-fA-F]{4})
space ::= | " " | "\n" [ \t]{0,20}
string ::= "\"" char* "\"" space
listofstring ::= ("[" space (string ("," space string){4})? "]")
'''

# Creating a LlamaGrammar object with schema string
# Set verbose=False to not print the grammar, set to True for debugging
grammar_food_name = LlamaGrammar.from_string(grammar=schema_food_name, verbose=False)

In [10]:
result = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs in JSON.",
        },
        {"role": "user", "content": ask_food_prompt},
    ],
    grammar=grammar_food_name,
    temperature=0.7,
)

llama_perf_context_print:        load time =     787.70 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    63 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    70 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    7069.70 ms /   133 tokens


In [11]:
result_json = json.loads(result['choices'][0]['message']['content'])
result_json

{'food_names': ['Vegan Bibimbap',
  'Korean-Style BBQ Jackfruit',
  'Vegan Japchae (Stir-Fried Glass Noodles)',
  'Kimchi Stew (Kimchi Jjigae)',
  'Vegan Bulgogi (Marinated Vegan Beef)']}

In [12]:
query_res = [{"index":0,"query":"kimichi stew","Name":"Perfect Chicken Stew","RecipeInstructions":"c(\"Season a 3-7 pound chicken with Garlic powder and Pepper. Roast chicken in oven at 325 degrees.\", \"While chicken is cooking, dice potatoes, slice carrots, chop onions and carrots to desired thickness. Place vegetables in stewing pot and add water until vegetables are covered with about an 3 inches of water. Boil rapidly until potatoes are just finished.\", \"Remove vegetables from the pot by straining them and keep the water. By removing the vegetables and letting them cool, you prevent overcooking them and they won't dissolve into nothing.\", \n\"With remaining water on low heat, add can of cream of mushroom soup, can of chicken stock and milk (milk optional, Zie Ga Zink).\", \"If you don't use milk, I suggest a premium ready to serve brand of creamed mushroom soup, it will be of a smoother, creamier consistency than the regular cans of mushroom soup.\", \"Get a small sealable container and fill with 1 cup of cold water, then add 1 cup of flour, cover and seal, then immediately shake vigorously. You are making a thickener for the stew, it should look like the consistency of glue with no lumps. If to thick add a bit of water, too thin add a bit more flour, shake very hard again. If there are a few lumps you can remove them by straining. This process, once learned, is very useful for making gravies or other stews without using a high-fat butter and flour 'roux' thickener.\", \n\"Rapidly add thickener to the starch water/mushroom soup/stock/milk mixture using a whisk. You may have to make a little more thickener if you want a hardier stew, just remember that the stew will thicken more after it is removed from the heat and it stands. Simmer to desired consistency. Stir often. Do not burn! I suggest a non-stick stew pot, it helps prevent burning.\", \"Add the cooked (now cooled) vegetables to the stew.\", \"When chicken is finished roasting, drain juices into the stew. Remove skin and bones.  Tear or cut chicken apart and add to the stew.\", \n\"Stir in about 2-3 tablespoons of salt to stew  and about the same amount of pepper to taste.\", \"If you want, try adding a dash of hot sauce or a pinch of Sambel Olek.\", \"Let stew simmer for a little longer. Serve with fresh bread and Enjoy.\", \"Questions? brennarlauterbach@hotmail.com.\")"},{"index":1,"query":"kimichi stew","Name":"Wintry Beef Vegetable Stew With Fluffy Herb Dumplings","RecipeInstructions":"c(\"Cook and stir beef in shortening in heavy 8-10 quart stock pot, until beef is well browned. (Note: If too much liquid builds up to prevent adequate browning, pour off excess liquid into a bowl and reserve. Continue to brown the beef and when well browned, add the reserved liquid back into the pot.).\", \"Add 5 cups hot water, 1/2 teaspoon salt and the black pepper.\", \"Heat to boiling; reduce heat.\", \"Cover and simmer until beef is almost tender, 45 minutes to 1 hour.\", \"Stir in potato, turnip, rutabaga, carrots, green pepper, green beans (if using), celery, onion, bouquet sauce, the bouillon cube and bay leaves.\", \n\"Cover and simmer until vegetables are tender (but do not overcook), stirring once, about 25 minutes.\", \"Prepare dough (see below) for Dumplings;  set aside.\", \"Using a fork, blend together 1 cup cold water and the 4 tablespoons flour in a small mixing bowl; stir gradually into stew.\", \"Heat to boiling, stirring constantly.\", \"Boil and stir 1 minute; reduce heat.\", \"Do ahead tip: After boiling and stirring 1 minute, stew can be covered and refrigerated no longer than 48 hours. To serve, heat to boiling over medium-high heat. Continue as directed.\", \n\"DUMPLINGS:\", \"In a large bowl, cut shortening into combined flour, baking powder, salt, parsley and herbs until mixture resembles fine crumbs.\", \"Stir in milk.\", \"Drop by heaping tablespoons onto hot meat or vegetables in boiling stew (do not drop directly into liquid).\", \"Cook uncovered 15 minutes.\", \"Cover and cook about 15 minutes longer. Cut a dumpling in half to test for doneness; you want them done but not dry!\", \"Serve stew piping hot, with a buttered baguette and a glass of cider, ale, or wine. As with all good stews, this stew is even better reheated the next day, after flavors have had a chance to meld. Stew leftovers freeze and reheat beautifully, and would make a delicious cottage or shepherd's pie.\"\n)"}]

In [13]:
import re
extracted_recipe = {}
for recipe in query_res:
    recipe_name = recipe['Name']
    recipe_instructions = recipe["RecipeInstructions"]
    matches = re.findall(r'"(.*?)"', recipe_instructions)
    recipe_instructions = tuple(matches)
    steps_dict = {f"Step {i+1}": step for i, step in enumerate(recipe_instructions)}
    steps_json = json.dumps(steps_dict, indent=2)
    extracted_recipe[recipe_name] = {} # Overwrite recipe with same name, no duplicate
    extracted_recipe[recipe_name]['instructions'] = steps_json

In [14]:
ask_recipe_template = "The user is living in {user_location}. The user is looking for a {user_query}. Here are few potential recipes: {retrieved_recipes}. Based on such recipe, generate a new recipe that satisfy user nutrition requirements. You can reuse the existing recipe, or you can modify it or create a new recipe."

In [15]:
ask_recipe_prompt = ask_recipe_template.format(user_location=user_location, user_query=user_query, retrieved_recipes=extracted_recipe)

In [16]:
# schema_recipe = r'''
# root ::= (
#     "{" newline
#         doublespace "\"Description\":" space string "," newline
#         doublespace "\"Cooking instructions\":" cookinstructs "," space 
#         # doublespace "\"Ingredients\":" space ingreds newline
#     "}"
# )
# newline ::= "\n"
# doublespace ::= "  "
# number ::= [0-9]+   "."?   [0-9]*
# integer ::= [0-9]*
# boolean ::= "true" | "false"
# char ::= [^"\\\x7F\x00-\x1F] | [\\] (["\\bfnrt] | "u" [0-9a-fA-F]{4})
# space ::= | " " | "\n" [ \t]{0,20}
# string ::= "\"" char* "\"" space
# sentence ::= char* space
# listofstring ::= ("[" space (string ("," space string)*)? "]")
# cookstep ::= ("\"Step" space integer "\"" ":" space string)
# cookinstructs ::= (space "{" space (cookstep ("," space cookstep){20})? "}")
# ingred ::= ("{" space "\"Ingredient\":" space string ", \"Unit\": " string ", \"Amount\": " number "}")
# ingreds ::= (space "\[" space (ingred ("," space ingred){0,30}) "\]")
# '''


schema_recipe = r'''
root ::= (
    "{" newline
        doublespace "\"Description\":" space string "," newline
        doublespace "\"Cooking instructions\":" cookinstructs space 
    "}"
)
newline ::= "\n"
doublespace ::= "  "
number ::= [0-9]+   "."?   [0-9]*
integer ::= [0-9]*
boolean ::= "true" | "false"
char ::= [^"\\\x7F\x00-\x1F] | [\\] (["\\bfnrt] | "u" [0-9a-fA-F]{4})
space ::= | " " | "\n" [ \t]{0,20}
string ::= "\"" char* "\"" space
sentence ::= char* space
listofstring ::= ("[" space (string ("," space string)*)? "]")
cookstep ::= ("\"Step" space integer "\"" ":" space string)
cookinstructs ::= (space "{" space (cookstep ("," space cookstep){10})? "}")
'''

grammar_recipe = LlamaGrammar.from_string(grammar=schema_recipe, verbose=False)

In [17]:
result = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs in JSON. You must not include nutrition information or any notes. Stop generating instructions when the food is ready to serve. Do not put notes or nutrition in cooking instructions. Your output will be in the format: {\"Description\": A brief description of the recipe, \"Cooking Instructions\": {\"Step 1\": first cooking step, \"Step 2\": second cooking step, ...}}",
        },
        {"role": "user", "content": ask_recipe_prompt},
    ],
    grammar=grammar_recipe,
    temperature=0.7,
    # stream=True
)

# for chunk in result:
#     delta = chunk['choices'][0]['delta']
#     if 'role' in delta:
#         print(delta['role'], end=': ')
#     elif 'content' in delta:
#         print(delta['content'], end='')


Llama.generate: 15 prefix-match hit, remaining 1404 prompt tokens to eval
llama_perf_context_print:        load time =     787.70 ms
llama_perf_context_print: prompt eval time =       0.00 ms /  1404 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   396 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   61289.88 ms /  1800 tokens


In [18]:
print(result['choices'][0]['message']['content'])

{
  "Description": "Vegan Korean-Style Stew",
  "Cooking instructions": {
    "Step 1": "Cook and stir vegan beef strips in a little oil in a large pot until browned. Add 5 cups of vegetable broth, 1/2 teaspoon of salt, and the black pepper. Heat to boiling; reduce heat.",
    "Step 2": "Add 1 cup of diced Korean chili flakes (gochugaru), 1 cup of diced carrots, 1 cup of diced zucchini, 1 cup of diced bell peppers, and 1 cup of diced mushrooms. Simmer until the vegetables are tender.",
    "Step 3": "Prepare the Korean chili paste (gochujang) by mixing 2 tablespoons of gochujang with 2 tablespoons of soy sauce and 2 tablespoons of rice vinegar.",
    "Step 4": "Add the gochujang mixture to the pot and stir to combine. Simmer for 10 minutes.",
    "Step 5": "Serve the stew hot, garnished with green onions and crispy garlic. Enjoy!"
  							 	,"Step 6": "Optional: Serve with a side of steamed rice or noodles."
  							 	,"Step 7": "Optional: Add some kimchi to the stew for an extra kic

In [19]:
cook_steps = json.loads(result['choices'][0]['message']['content'])

In [20]:
concat_steps = cook_steps['Description']
concat_steps += '\n'

for step in cook_steps['Cooking instructions']:
    # concat_steps.append()
    concat_steps += step
    concat_steps += ': '
    concat_steps += cook_steps['Cooking instructions'][step]
    concat_steps += '\n'
concat_steps = concat_steps[:-1]

In [21]:
print(concat_steps)

Vegan Korean-Style Stew
Step 1: Cook and stir vegan beef strips in a little oil in a large pot until browned. Add 5 cups of vegetable broth, 1/2 teaspoon of salt, and the black pepper. Heat to boiling; reduce heat.
Step 2: Add 1 cup of diced Korean chili flakes (gochugaru), 1 cup of diced carrots, 1 cup of diced zucchini, 1 cup of diced bell peppers, and 1 cup of diced mushrooms. Simmer until the vegetables are tender.
Step 3: Prepare the Korean chili paste (gochujang) by mixing 2 tablespoons of gochujang with 2 tablespoons of soy sauce and 2 tablespoons of rice vinegar.
Step 4: Add the gochujang mixture to the pot and stir to combine. Simmer for 10 minutes.
Step 5: Serve the stew hot, garnished with green onions and crispy garlic. Enjoy!
Step 6: Optional: Serve with a side of steamed rice or noodles.
Step 7: Optional: Add some kimchi to the stew for an extra kick of flavor.
Step 8: Optional: Use different types of mushrooms or vegetables to change up the flavor and texture of the stew

In [28]:
ask_ingreds_template = "The user is living in {user_location}. The user is looking for a {user_query}. Here is a generated recipe: {generated_recipes}. Generate all ingredients with their corresponding unit and amount in the recipe. For each ingredient, list their name, and their weight in grams."

ask_ingreds_prompt = ask_ingreds_template.format(user_location=user_location, user_query=user_query, generated_recipes=concat_steps)

In [29]:
print(ask_ingreds_prompt)

The user is living in California. The user is looking for a vegan korean food. Here is a generated recipe: Vegan Korean-Style Stew
Step 1: Cook and stir vegan beef strips in a little oil in a large pot until browned. Add 5 cups of vegetable broth, 1/2 teaspoon of salt, and the black pepper. Heat to boiling; reduce heat.
Step 2: Add 1 cup of diced Korean chili flakes (gochugaru), 1 cup of diced carrots, 1 cup of diced zucchini, 1 cup of diced bell peppers, and 1 cup of diced mushrooms. Simmer until the vegetables are tender.
Step 3: Prepare the Korean chili paste (gochujang) by mixing 2 tablespoons of gochujang with 2 tablespoons of soy sauce and 2 tablespoons of rice vinegar.
Step 4: Add the gochujang mixture to the pot and stir to combine. Simmer for 10 minutes.
Step 5: Serve the stew hot, garnished with green onions and crispy garlic. Enjoy!
Step 6: Optional: Serve with a side of steamed rice or noodles.
Step 7: Optional: Add some kimchi to the stew for an extra kick of flavor.
Step 

In [30]:
schema_recipe = r'''
root ::= (
    "{" newline
        doublespace "\"Ingredients\":" space ingreds newline
    "}"
)
newline ::= "\n"
doublespace ::= "  "
number ::= [0-9]+   "."?   [0-9]*
integer ::= [0-9]*
boolean ::= "true" | "false"
char ::= [^"\\\x7F\x00-\x1F] | [\\] (["\\bfnrt] | "u" [0-9a-fA-F]{4})
space ::= | " " | "\n" [ \t]{0,20}
string ::= "\"" char* "\"" space
sentence ::= char* space
listofstring ::= ("[" space (string ("," space string)*)? "]")
cookstep ::= ("\"Step" space integer "\"" ":" space string)
cookinstructs ::= (space "{" space (cookstep ("," space cookstep){10})? "}")
ingredname ::= ("\"Ingredient name" space "\"" ":" space string)
ingredunit ::= ("\"Unit" space "\"" ":" space string)
ingredamt ::= ("\"Amount" space "\"" ":" space number)
ingredwgt ::= ("\"Grams" space "\"" ":" space number)
ingredset ::= ("{" ingredname "," space ingredwgt space "}")
ingreds ::= (space "[" space (ingredset ("," space ingredset){20})? "]")
'''

grammar_recipe = LlamaGrammar.from_string(grammar=schema_recipe, verbose=False)


result = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs in JSON. You must not include nutrition information or any notes. Do not put notes or nutrition in cooking instructions.",
        },
        {"role": "user", "content": ask_ingreds_prompt},
    ],
    grammar=grammar_recipe,
    temperature=0.7,
    # stream=True
)

# for chunk in result:
#     delta = chunk['choices'][0]['delta']
#     if 'role' in delta:
#         print(delta['role'], end=': ')
#     elif 'content' in delta:
#         print(delta['content'], end='')


Llama.generate: 404 prefix-match hit, remaining 19 prompt tokens to eval
llama_perf_context_print:        load time =     787.70 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    19 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   366 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   39242.54 ms /   385 tokens


In [31]:
ingreds = json.loads(result['choices'][0]['message']['content'])

In [32]:
ingreds['Ingredients']

[{'Ingredient name': 'Vegan beef strips', 'Grams': 200},
 {'Ingredient name': 'Oil', 'Grams': 30},
 {'Ingredient name': 'Vegetable broth', 'Grams': 1100},
 {'Ingredient name': 'Salt', 'Grams': 6},
 {'Ingredient name': 'Black pepper', 'Grams': 3},
 {'Ingredient name': 'Korean chili flakes (gochugaru)', 'Grams': 450},
 {'Ingredient name': 'Carrots', 'Grams': 250},
 {'Ingredient name': 'Zucchini', 'Grams': 250},
 {'Ingredient name': 'Bell peppers', 'Grams': 250},
 {'Ingredient name': 'Mushrooms', 'Grams': 250},
 {'Ingredient name': 'Gochujang', 'Grams': 120},
 {'Ingredient name': 'Soy sauce', 'Grams': 60},
 {'Ingredient name': 'Rice vinegar', 'Grams': 60},
 {'Ingredient name': 'Green onions', 'Grams': 30},
 {'Ingredient name': 'Crispy garlic', 'Grams': 30},
 {'Ingredient name': 'Sesame oil', 'Grams': 15},
 {'Ingredient name': 'Sesame seeds', 'Grams': 15},
 {'Ingredient name': 'Kimchi', 'Grams': 150},
 {'Ingredient name': 'Steamed rice', 'Grams': 200},
 {'Ingredient name': 'Noodles', 'Gram

In [33]:
ingreds_concat = ""
for ing in ingreds['Ingredients']:
    ingreds_concat += ing['Ingredient name']
    ingreds_concat += ' '
    ingreds_concat += str(ing['Amount'])
    ingreds_concat += ' '
    ingreds_concat += ing['Unit']
    ingreds_concat += '.\n'
print(ingreds_concat)

KeyError: 'Amount'

In [34]:
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import process

/fs01/home/lfy/attribution/context-attribution/contatt/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [40]:
from fuzzywuzzy import process
# Function to get the best match from df2 for each ingredient
def get_best_match(ingredient, choices, threshold=80):
    best_match, score = process.extractOne(ingredient, choices)
    if score >= threshold:
        return best_match
    return None  # If no good match is found

In [41]:
df_nutritions = pd.read_csv("nutrition.csv")
num_nutrition_with_missing_info = df_nutritions.isnull().any(axis=1).sum()
print(f"Number of recipes with missing info: {num_nutrition_with_missing_info}")
missing_per_column_nutrition = df_nutritions.isnull().sum()
missing_per_column_nutrition = missing_per_column_nutrition[missing_per_column_nutrition > 0]
print(f"Missing values per column:\n{missing_per_column_nutrition}")
df_nutritions.fillna(0, inplace=True)

Number of recipes with missing info: 1590
Missing values per column:
saturated_fat    1590
dtype: int64


In [42]:
important_nutrients = ['calories', 'total_fat', 'fat', 'saturated_fat', 'saturated_fatty_acids', 'cholesterol',
 'sodium', 'carbohydrate', 'fiber', 'sugars', 'protein', 'vitamin_a', 'vitamin_a_rae',
 'carotene_alpha', 'carotene_beta', 'cryptoxanthin_beta', 'lutein_zeaxanthin', 'lucopene',
 'vitamin_b12', 'vitamin_b6', 'vitamin_c', 'vitamin_d', 'vitamin_e', 'tocopherol_alpha',
 'vitamin_k', 'calcium', 'copper', 'iron', 'magnesium', 'manganese', 'phosphorous',
 'potassium', 'selenium', 'zinc', 'water']

In [ ]:
def get_best_match(ingredients, df, important_nutrients, threshold=80):
    choices = df['name'].tolist()

    result = []
    for ingredient in ingredients:
        best_match, score = process.extractOne(ingredient['Ingredient name'], choices)

        # if the score doesn't reach threshold, there is no valid match found for ingredient
        if score < threshold:
            print(f"No valid match found for {ingredient['Ingredient name']}")
            # put 0 for all important nutrients
            for important_nutrient in important_nutrients:
                ingredient[important_nutrient] = 0
            result.append(ingredient)
            continue

        # combine the columns in df['name'] == best_match to key and value pairs in the dictionary
        # only add the key and value pairs where key exists in important_nutrients
        best_match_row = df[df['name'] == best_match].to_dict('records')[0]
        for key, value in best_match_row.items():
            if key in important_nutrients:

                if isinstance(value, str):  # If the value is a string, check for units
                    # Use regular expression to remove units like 'g', 'mg', 'IU', etc.
                    cleaned_value = re.sub(r'[^\d.-]', '', value)  # Remove anything that's not a digit or decimal point
                    # add value to ingredient
                    ingredient[key] = float(cleaned_value) * float(ingredient['Grams']) / 100 if cleaned_value else 0.0

        result.append(ingredient)

    return result


In [56]:
recipe_with_nutrients = get_best_match(ingreds['Ingredients'], df_nutritions, important_nutrients=important_nutrients)
for ingredient in recipe_with_nutrients:
    print(ingredient)

{'Ingredient name': 'Vegan beef strips', 'Grams': 200}
{'Ingredient name': 'Oil', 'Grams': 30}
{'Ingredient name': 'Vegetable broth', 'Grams': 1100}
{'Ingredient name': 'Salt', 'Grams': 6}
{'Ingredient name': 'Black pepper', 'Grams': 3}
{'Ingredient name': 'Korean chili flakes (gochugaru)', 'Grams': 450}
{'Ingredient name': 'Carrots', 'Grams': 250}
{'Ingredient name': 'Zucchini', 'Grams': 250}
{'Ingredient name': 'Bell peppers', 'Grams': 250}
{'Ingredient name': 'Mushrooms', 'Grams': 250}
{'Ingredient name': 'Gochujang', 'Grams': 120}
No valid match found for Gochujang
{'Ingredient name': 'Soy sauce', 'Grams': 60}
{'Ingredient name': 'Rice vinegar', 'Grams': 60}
{'Ingredient name': 'Green onions', 'Grams': 30}
{'Ingredient name': 'Crispy garlic', 'Grams': 30}
{'Ingredient name': 'Sesame oil', 'Grams': 15}
{'Ingredient name': 'Sesame seeds', 'Grams': 15}
{'Ingredient name': 'Kimchi', 'Grams': 150}
{'Ingredient name': 'Steamed rice', 'Grams': 200}
{'Ingredient name': 'Noodles', 'Grams': 

In [63]:
from collections import defaultdict

In [65]:
nutrients_dict = defaultdict(float)

In [66]:
for ing in recipe_with_nutrients:
    for nu in important_nutrients:
        ing[nu]

KeyError: 'calories'

In [61]:
ing

{'Ingredient name': 'Rice',
 'Grams': 200,
 'total_fat': 0.4,
 'cholesterol': 0.0,
 'sodium': 38.0,
 'vitamin_a': 0.0,
 'vitamin_a_rae': 0.0,
 'carotene_alpha': 0.0,
 'carotene_beta': 0.0,
 'cryptoxanthin_beta': 0.0,
 'lutein_zeaxanthin': 0.0,
 'vitamin_b12': 0.0,
 'vitamin_b6': 0.012,
 'vitamin_c': 0.0,
 'vitamin_d': 0.0,
 'vitamin_e': 0.06,
 'tocopherol_alpha': 0.06,
 'vitamin_k': 0.0,
 'calcium': 8.0,
 'copper': 0.076,
 'magnesium': 6.0,
 'manganese': 0.228,
 'phosphorous': 40.0,
 'potassium': 8.0,
 'selenium': 9.0,
 'protein': 3.58,
 'carbohydrate': 48.02,
 'fiber': 2.0,
 'sugars': 0.06,
 'fat': 0.4,
 'saturated_fatty_acids': 0.046,
 'water': 147.64}

In [ ]:
eval_prompt_template = "The user is living in {user_location}. The user is looking for a {user_query}. Here is a generated recipe: {generated_recipes}. Generate all ingredients with their corresponding unit and amount in the recipe."

eval_prompt = eval_prompt_template.format(user_location=user_location, user_query=user_query, generated_recipes=concat_steps)

In [ ]:
schema_eval = r'''
root ::= (
    "{" newline
        doublespace "\"Results\":" space boolean "," newline
        doublespace "\"Reasons\":" space string newline
    "}"
)
newline ::= "\n"
doublespace ::= "  "
number ::= [0-9]+   "."?   [0-9]*
integer ::= [0-9]*
boolean ::= "true" | "false"
char ::= [^"\\\x7F\x00-\x1F] | [\\] (["\\bfnrt] | "u" [0-9a-fA-F]{4})
space ::= | " " | "\n" [ \t]{0,20}
string ::= "\"" char* "\"" space
sentence ::= char* space
listofstring ::= ("[" space (string ("," space string)*)? "]")
cookstep ::= ("\"Step" space integer "\"" ":" space string)
cookinstructs ::= (space "{" space (cookstep ("," space cookstep){10})? "}")
ingredname ::= ("\"Ingredient name" space "\"" ":" space string)
ingredunit ::= ("\"Unit" space "\"" ":" space string)
ingredamt ::= ("\"Amount" space "\"" ":" space number)
ingredset ::= ("{" ingredname "," space ingredunit "," space ingredamt space "}")
ingreds ::= (space "[" space (ingredset ("," space ingredset){20})? "]")
'''

grammar_recipe = LlamaGrammar.from_string(grammar=schema_recipe, verbose=False)


result = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that outputs in JSON. You are provided with a recipe and its corresponding nutrition. Evaluate whether the nutrition is following the requirements.",
        },
        {"role": "user", "content": eval_prompt},
    ],
    grammar=grammar_recipe,
    temperature=0.7,
    stream=True
)

for chunk in result:
    delta = chunk['choices'][0]['delta']
    if 'role' in delta:
        print(delta['role'], end=': ')
    elif 'content' in delta:
        print(delta['content'], end='')
